In [66]:
#Dataframe creation by Bryan Miller
#Clean and Merge National Obesity by State and covid deaths 
import pandas as pd
from pathlib import Path
import numpy as np
from pprint import pprint
import sqlite3

csv_path = Path("../Resources/National_Obesity_By_State.csv")
Obease_df = pd.read_csv(csv_path, encoding="utf-8")

csv_path1 = Path("../Resources/states.csv")
States = pd.read_csv(csv_path1, encoding="utf-8")

csv2 = Path("../Resources/covid_deaths_usafacts.csv")
CD_df = pd.read_csv(csv2, encoding="utf-8")

csv3 = Path("../Resources/covid_county_population_usafacts.csv")
cp_df = pd.read_csv(csv3, encoding="utf_8")


Create Dataframe

In [67]:
# Original Data frame
Obease_df.head()



,FID,State,Obesity,SHAPE_Length,SHAPE_Area
0,1,Texas,32.4,1.540832e+07,7.670000e+12
1,2,California,24.2,1.451870e+07,5.330000e+12
2,3,Kentucky,34.6,6.346699e+06,1.130000e+12
3,4,Georgia,30.7,5.795596e+06,1.650000e+12
4,5,Wisconsin,30.7,6.806782e+06,1.570000e+12


In [68]:
States.head()


,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Get abreviation for each state

In [69]:
Obease_merge = States.merge(Obease_df,on='State')
Obease_merge.head()

,State,Abbreviation,FID,Obesity,SHAPE_Length,SHAPE_Area
0,Alabama,AL,38,35.6,5.750658e+06,1.440000e+12
1,Alaska,AK,14,29.8,6.663727e+06,5.300000e+11
2,Arizona,AZ,43,28.4,8.044184e+06,3.560000e+12
3,Arkansas,AR,24,34.5,5.707634e+06,1.490000e+12
4,California,CA,2,24.2,1.451870e+07,5.330000e+12


In [70]:
#Merge databases pull out the unessecary columns
Obease_merge = Obease_merge[['Abbreviation','Obesity']]
Obease_merge1 = Obease_merge.set_axis(['State', 'Obesity'], axis='columns', inplace=False)
Obease_merge1.head()



C:\Users\Bryan\AppData\Local\Temp\ipykernel_31304\210589381.py:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  Obease_merge1 = Obease_merge.set_axis(['State', 'Obesity'], axis='columns', inplace=False)


,State,Obesity
0,AL,35.6
1,AK,29.8
2,AZ,28.4
3,AR,34.5
4,CA,24.2


In [71]:
CD_df.head()


,countyFIPS,County Name,State,StateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,7/14/2023,7/15/2023,7/16/2023,7/17/2023,7/18/2023,7/19/2023,7/20/2023,7/21/2023,7/22/2023,7/23/2023
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,235,235,235,235,235,235,235,235,235,235
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,731,731,731,731,731,731,731,731,731,731
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,104,104,104,104,104,104,104,104,104,104
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,111,111,111,111,111,111,111,111,111,111


In [72]:
#remove uneeded columns
Final_Deaths = CD_df[['State','7/23/2023']]
#Change column Name
Final_Deaths.rename(columns={'7/23/2023':'Covid Deaths'}, inplace=True)


# Groupby state and Sum
State_Deaths = Final_Deaths.groupby(Final_Deaths['State']).sum()
State_Deaths.head()





C:\Users\Bryan\AppData\Local\Temp\ipykernel_31304\3697702902.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Deaths.rename(columns={'7/23/2023':'Covid Deaths'}, inplace=True)


,Covid Deaths
State,
AK,1457
AL,21138
AR,13062
AZ,29852
CA,102356


In [73]:
#POPULATION by county for covid information
cp_df.head()
Pop = cp_df[['State','population']]


In [74]:
State_population = Pop.groupby(Pop['State']).sum()
State_population.head()

,population
State,
AK,731545
AL,4903185
AR,3017804
AZ,7278717
CA,39512223


# Merge the 3 data frames by state
State_population - State_Deaths  -  Obease


In [75]:
Merged_DF = State_population.merge(State_Deaths,on='State')


In [76]:
Merged_DF['%Deaths'] = Merged_DF['Covid Deaths'] / Merged_DF['population']


In [77]:
Final = Merged_DF.merge(Obease_merge1,on='State')
Final.head()

,State,population,Covid Deaths,%Deaths,Obesity
0,AK,731545,1457,0.001992,29.8
1,AL,4903185,21138,0.004311,35.6
2,AR,3017804,13062,0.004328,34.5
3,AZ,7278717,29852,0.004101,28.4
4,CA,39512223,102356,0.002590,24.2


In [78]:
Final.to_csv("../Resources/Final_Obesity_Covid.csv",
                  encoding="utf-8", index=False, header=True)

In [79]:
json = Final.to_json("../Resources/Final_Obesity_Covid.json")

In [80]:
pprint(json)


None
